In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/geiot

/content/drive/.shortcut-targets-by-id/1HvjC1Sh_DFmMvqXUWGoYm1lsmbFoWO8K/Colab Notebooks/geiot


In [ ]:
!conda install matplotlib

/bin/bash: conda: command not found


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from tqdm import tqdm
from torchvision import models,transforms
from PIL import Image
import cv2
import sys
import json
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
import os
import os.path as osp
from glob import glob
import pandas as pd

In [ ]:
torch.__version__

'1.9.0'

In [ ]:
torchvision.__version__

'0.10.0'

# データのフォルダーを分ける

Root----------input<br>
｜&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;｜<br>
｜&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;｜ー&nbsp;train&nbsp;ーimages<br>
｜&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;｜ー&nbsp;test&nbsp;ーimages<br>
｜<br>
&nbsp;&nbsp;ーorignal_data(KDEF)


In [ ]:
Root="TeamB"
input_dir=osp.join(Root,"input")
os.makedirs(input_dir,exist_ok=True)
train_dir=osp.join(input_dir,"train")
os.makedirs(train_dir,exist_ok=True)
test_dir=osp.join(input_dir,"test")
os.makedirs(test_dir,exist_ok=True)

In [ ]:
original_data_dir="TeamB/KDEF"
image_paths=glob(osp.join(original_data_dir,"*","*.JPG"))
#print(image_paths)
image_label_dict={"imagePath":[],"Label":[]}
for path in image_paths:
  #basenaemは拡張子を含んだファイル名,splitextは「.」で分割するので[0]を指定すると、拡張子なしのファイル名が手に入る
  label= osp.splitext(os.path.basename(path))[0]
  if not ((label[4:6]) in ["AF","AN","DI","HA","NE","SA","SU"]):
    continue
  image_label_dict["Label"].append(label[4:6])
  image_label_dict["imagePath"].append(path)

In [ ]:
db_path_label=pd.DataFrame.from_dict(image_label_dict)

In [ ]:
len(db_path_label)

4898

In [ ]:
db_path_label.Label.unique()

array(['AF', 'AN', 'DI', 'HA', 'NE', 'SA', 'SU'], dtype=object)

In [ ]:
db_path_label

,imagePath,Label
0,TeamB/KDEF\AF01\AF01AFFL.JPG,AF
1,TeamB/KDEF\AF01\AF01AFFR.JPG,AF
2,TeamB/KDEF\AF01\AF01AFHL.JPG,AF
3,TeamB/KDEF\AF01\AF01AFHR.JPG,AF
4,TeamB/KDEF\AF01\AF01AFS.JPG,AF
...,...,...
4893,TeamB/KDEF\BM35\BM35SUFL.JPG,SU
4894,TeamB/KDEF\BM35\BM35SUFR.JPG,SU
4895,TeamB/KDEF\BM35\BM35SUHL.JPG,SU
4896,TeamB/KDEF\BM35\BM35SUHR.JPG,SU


In [ ]:
#AF = afraid,AN = angry,DI = disgusted,HA = happy,NE = neutral,SA = sad,SU = surprised
replace_dict={"AF":0,"AN":1,"DI":2,"HA":3,"NE":4,"SA":5,"SU":6}
db_path_label.Label=db_path_label.Label.map(replace_dict)
db_path_label.imagePath[db_path_label.Label!=5]

0       TeamB/KDEF\AF01\AF01AFFL.JPG
1       TeamB/KDEF\AF01\AF01AFFR.JPG
2       TeamB/KDEF\AF01\AF01AFHL.JPG
3       TeamB/KDEF\AF01\AF01AFHR.JPG
4        TeamB/KDEF\AF01\AF01AFS.JPG
                    ...             
4893    TeamB/KDEF\BM35\BM35SUFL.JPG
4894    TeamB/KDEF\BM35\BM35SUFR.JPG
4895    TeamB/KDEF\BM35\BM35SUHL.JPG
4896    TeamB/KDEF\BM35\BM35SUHR.JPG
4897     TeamB/KDEF\BM35\BM35SUS.JPG
Name: imagePath, Length: 4199, dtype: object

In [ ]:
train_data,test_data=train_test_split(db_path_label,test_size=1000)

In [ ]:
train_data = train_data.reset_index(drop=True)
test_data=test_data.reset_index(drop=True)

In [ ]:
for i,j in zip(train_data.imagePath,test_data.imagePath):
  train_img=cv2.imread(i)
  test_img=cv2.imread(j)
  cv2.imwrite(osp.join(train_dir,osp.basename(i)),train_img)
  cv2.imwrite(osp.join(test_dir,osp.basename(j)),test_img)
train_data.to_csv(osp.join(train_dir,"train_anno.csv"))
test_data.to_csv(osp.join(test_dir,"test_anno.csv"))